### Imports

In [2]:
import os
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from rembg import remove as rembg_remove
from pathlib import Path
from glob import glob
from tqdm import tqdm

### Config

In [ ]:
BASE_DIR = os.path.abspath("../").replace("\\", "/")

LEAF_BASE = f"{BASE_DIR}/dataset_bg_removed"
BG_DIR = f"{BASE_DIR}/background_images"
OUT_BASE = f"{BASE_DIR}/augmented-dataset"

print("LEAF_DIR:", LEAF_BASE)
print("BG_DIR:", BG_DIR)
print("OUT_DIR:", OUT_BASE)

TARGET_WIDTH = 600
AUG_PER_LEAF = 2

### Functions

In [4]:
def resize_with_aspect_ratio(image, target_width):
    h, w = image.shape[:2]
    scale = target_width / w
    target_height = int(h * scale)
    return cv2.resize(image, (target_width, target_height))

In [5]:
def random_affine(image):
    rows, cols = image.shape[:2]
    tx = random.uniform(-0.1, 0.1) * cols
    ty = random.uniform(-0.1, 0.1) * rows
    scale = random.uniform(0.8, 1.2)
    angle = random.uniform(-15, 15)
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, scale)
    M[0, 2] += tx
    M[1, 2] += ty
    return cv2.warpAffine(image, M, (cols, rows))

In [6]:
def random_perspective(image):
    rows, cols = image.shape[:2]
    pts1 = np.float32([[0, 0], [cols-1, 0], [cols-1, rows-1], [0, rows-1]])
    pts2 = np.float32([
        [random.uniform(0, cols*0.2), random.uniform(0, rows*0.2)],
        [random.uniform(cols*0.8, cols), random.uniform(0, rows*0.2)],
        [random.uniform(cols*0.8, cols), random.uniform(rows*0.8, rows)],
        [random.uniform(0, cols*0.2), random.uniform(rows*0.8, rows)]
    ])
    M = cv2.getPerspectiveTransform(pts1, pts2)
    return cv2.warpPerspective(image, M, (cols, rows))

In [7]:
def remove_background(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    lower_bound = np.array([25, 40, 40])
    upper_bound = np.array([90, 255, 255])

    mask = cv2.inRange(hsv, lower_bound, upper_bound)

    kernel = np.ones((3, 3), np.uint8)
    edges_dilated = cv2.dilate(mask, kernel, iterations=2)

    refined_mask = cv2.bitwise_or(mask, edges_dilated)

    result = cv2.bitwise_and(image, image, mask=refined_mask)
    alpha = refined_mask

    b, g, r = cv2.split(result)
    result = cv2.merge((b, g, r, alpha))

    return result

In [8]:
def overlay_image(background, overlay, position, scale=1.0, angle=0):
    overlay = rembg_remove(overlay)

    overlay = random_affine(overlay)
    overlay = random_perspective(overlay)

    h, w = overlay.shape[:2]
    new_w, new_h = int(w * scale), int(h * scale)
    overlay_resized = cv2.resize(overlay, (new_w, new_h))

    center = (new_w // 2, new_h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    overlay_resized = cv2.warpAffine(overlay_resized, M, (new_w, new_h))

    x, y = position
    h_bg, w_bg = background.shape[:2]

    if x + new_w > w_bg or y + new_h > h_bg:
        print(f"Overlay exceeds background dimensions. {x + new_w} > {w_bg} or {y + new_h} > {h_bg}")
        return background

    if overlay_resized.shape[2] == 4:
        alpha = overlay_resized[:, :, 3] / 255.0
        overlay_resized = overlay_resized[:, :, :3]
    else:
        alpha = np.ones((new_h, new_w))

    for c in range(3):
        background[y:y+new_h, x:x+new_w, c] = (
            background[y:y+new_h, x:x+new_w, c] * (1 - alpha) +
            overlay_resized[:, :, c] * alpha
        )

    return background

### Main

In [13]:
bg_paths = glob(f"{BG_DIR}/*.jpg") + glob(f"{BG_DIR}/*.png") + glob(f"{BG_DIR}/*.JPG")

os.makedirs(OUT_BASE, exist_ok=True)

In [ ]:
for split in ["train", "test"]:
    split_dir = os.path.join(LEAF_BASE, split)
    print("SPLIT DIR", split_dir)
    for class_dir in os.listdir(split_dir):
        class_path = os.path.join(split_dir, class_dir)
        print("CLASS PATH", class_path)
        leaf_paths = glob(f"{class_path}/*.png") + glob(f"{class_path}/*.JPG")
        print(len(leaf_paths))
        output_class_dir = os.path.join(OUT_BASE, split, class_dir)
        os.makedirs(output_class_dir, exist_ok=True)
        
        print(f"Processing {len(leaf_paths)} images from {split}/{class_dir}")

        for leaf_idx, leaf_path in enumerate(tqdm(leaf_paths, desc=f"{split}/{class_dir}")):
            leaf = cv2.imread(leaf_path, cv2.IMREAD_UNCHANGED)
            if leaf is None:
                continue

            for i in range(AUG_PER_LEAF):
                bg_path = random.choice(bg_paths)
                bg = cv2.imread(bg_path, cv2.IMREAD_UNCHANGED)
                if bg is None:
                    continue

                bg = resize_with_aspect_ratio(bg, TARGET_WIDTH)

                scale = random.uniform(1.5, 2.25)
                new_w = int(leaf.shape[1] * scale)
                new_h = int(leaf.shape[0] * scale)

                max_x = max(0, bg.shape[1] - new_w)
                max_y = max(0, bg.shape[0] - new_h)
                x = random.randint(0, max_x)
                y = random.randint(0, max_y)
                angle = random.randint(0, 360)

                composed = overlay_image(bg, leaf, (x, y), scale=scale, angle=angle)

                leaf_name = Path(leaf_path).stem
                out_path = os.path.join(output_class_dir, f"{leaf_name}_aug{i+1}.png")
                cv2.imwrite(out_path, composed)